Related to user story: [SP11-Item04: General Data Wrapper PoC](https://gitlab.inria.fr/fedbiomed/fedbiomed/-/issues/164)

## Tabular dataset

Workflow of data pre processing:

1. Columns name should be shared with the researcher
2. Data format file to be filled by clinicians.
3. Specify if missing data are allowed for a given columns (Exception). The file will be used for data verification during FL pre-processing,
4. Outlier verification for quantitative data, continuous and discrete, and for dates (Critical warning),
5. Missing data imputation by local mean (or optional NN), or majority voting for discrete labels. Give warnings when missing data are found (for verification a posteriori).
6. Give critical warning when too many missing are found (>50%),
7. Verify that number of available data is greater then minimum required (Error)

Critical warnings have different levels of disclosure to the researcher (1) only the warning, 2) type of warning, 3) type of warning and column affected).

In [60]:
!pip install prettytable

In [2]:
#1. load  a single view dataset


import pandas as pd
import pprint
import csv
import numpy as np
from typing import List, Tuple, Union, Dict, Any, Iterator, Optional
import os
from tabulate import tabulate
import json



In [2]:
from enum import Enum, auto

class ExcelSignatures(Enum):
    XLSX = (b'\x50\x4B\x05\x06', 2, -22, 4)
    LSX1 = (b'\x09\x08\x10\x00\x00\x06\x05\x00', 0, 512, 8)
    LSX2 = (b'\x09\x08\x10\x00\x00\x06\x05\x00', 0, 1536, 8)
    LSX3 = (b'\x09\x08\x10\x00\x00\x06\x05\x00', 0, 2048, 8)
    
    def __init__(self, sig, whence, offset, size):
        self._sig = sig
        self._whence = whence
        self._offset = offset
        self._size = size

    @property 
    def signature(self) -> bytes:
        return self._sig
    
    @property
    def whence(self) -> int:
        return self._whence
    
    @property
    def offset(self) -> int:
        return self._offset
    
    @property
    def size(self) -> int:
        return self._size



def load_tabular_datasets(path:str) -> Dict[str, pd.DataFrame]:
    tabular_datasets = {}

    if os.path.isdir(path):
        print('directory found')
        _is_folder = True
        
        _tabular_data_files = os.listdir(path)
    else:
        print('file found')
        _is_folder = False
        _tabular_data_files = (path,)
        
    for tabular_data_file in _tabular_data_files:
        if _is_folder:
            tabular_data_file = os.path.join(path, tabular_data_file)
        
        _is_excel = excel_sniffer(tabular_data_file)
        _csv_delimiter, _csv_header = csv_sniffer(tabular_data_file)
        _view_name = os.path.basename(tabular_data_file)
        if _is_excel:
            tabular_datasets[_view_name] = load_excel_file(tabular_data_file)
        elif _csv_delimiter is not None:
            tabular_datasets[_view_name] = load_csv_file(tabular_data_file,
                                                               _csv_delimiter, 
                                                               _csv_header)
        else:
            print(f'warning: cannot parse {tabular_data_file}: not a tabular data file')
        
    return tabular_datasets

def load_csv_file(path:str, delimiter:str, header:int) -> pd.DataFrame:
    try:
        dataframe = pd.read_csv(path, delimiter=delimiter, header=header)
    except csv.Error as err:
        print('err', err, 'in file', path)
            
    return dataframe

#https://stackoverflow.com/questions/23515791/how-to-check-the-uploaded-file-is-csv-or-xls-in-python/23515973




def load_excel_file(path:str, sheet_name: Union[str, int]=0) -> pd.DataFrame:
    """May rely on openpyxl package"""
    #with open(path, 'r') as excl:
    #    _c = csv.DictReader(excl, dialect=csv.excel_tab)
    #    _delimiter = _c.dialect.delimiter
    
    dataframe = pd.read_excel(path, sheet_name=sheet_name)
    return dataframe


def excel_sniffer(path: str) -> bool:
    
    for excel_sig in ExcelSignatures:
        with open(path, 'rb') as f:
            f.seek(excel_sig.offset, excel_sig.whence)
            bytes = f.read(excel_sig.size)

            if bytes == excel_sig.signature:
                return True
            else:
                return False
            

def csv_sniffer(path:str) :
        
    with open(path, 'r') as csvfile:
        try:
            # do some operation on file using sniffer to make sure considered file
            # is a CSV file
            dialect = csv.Sniffer().sniff(csvfile.readline())
            delimiter = dialect.delimiter
            dialect.lineterminator
            has_header = csv.Sniffer().has_header(csvfile.readline())
            if has_header:
                header = 0
            else:
                header = None
        except (csv.Error, UnicodeDecodeError) as err:
            delimiter, header = None, None
            print('err', err, 'in file', path)
    return delimiter, header

In [52]:
!pip install openpyxl

In [30]:
load_tabular_datasets('../../Exceltest.xlsx')

file found
err 'utf-8' codec can't decode byte 0x8c in position 15: invalid start byte in file ../../Exceltest.xlsx


{'Exceltest.xlsx':    ID   Age Eligibility
 0    1   45           Y
 1    2   45           Y
 2    3   33           N
 3    4   54           Y
 4    5   45           Y
 5    6   54         NaN
 6    7   34           N
 7    8   54         NaN
 8    9   45         NaN
 9   10   44           Y}

In [11]:
single_view_dataset = load_tabular_datasets(r'/user/ybouilla/home/Documents/data/pseudo_adni_mod/pseudo_adni_mod.csv')

file found


In [81]:
multi_view_dataframe =  load_tabular_datasets('test7')
multi_view_dataframe

directory found


{'file1':      a   e   i   o      0      1      2      3                 time  pressure  \
 0   48  98  65   5  False   True  False  False  2018-01-01 00:00:00  0.088082   
 1   87  83  13  70   True  False   True  False  2018-01-01 01:00:00  0.774788   
 2   46  73  81  96  False  False  False   True  2018-01-01 02:00:00  0.514092   
 3   84  45  81  39  False   True   True   True  2018-01-01 03:00:00  0.832881   
 4   94  84   0  15  False   True  False  False  2018-01-01 04:00:00  0.696152   
 ..  ..  ..  ..  ..    ...    ...    ...    ...                  ...       ...   
 95  14  66  25  64   True   True   True  False  2018-01-04 23:00:00  0.295578   
 96  91  81  48  53  False  False   True   True  2018-01-05 00:00:00  0.474322   
 97  15  82  12  51   True   True   True   True  2018-01-05 01:00:00  0.927511   
 98  51  18   4  52  False  False   True   True  2018-01-05 02:00:00  0.494798   
 99  51  70  63  77  False  False   True  False  2018-01-05 03:00:00  0.316395   
 
     

Data format file to be filled by clinicians (step 2 int he workflow):

Data format file will be a dictionary specifying the type: 
* for single view datasets:
```{<feature_name>: {'data_type': <data_type>, 'type':<values_taken>, 'range': <value_range>}```
 * for multiview datatset
```{{<view_name>: <feature_name>: {'data_type': <data_type>, 'type':<values_taken>, 'range': <value_range>}}```

where
* `<view_name>` is the name of the view
* `<feature_name>` is the name of the feature
* `<data_type>` can be categorical or continuous or missing_data or datetime
* `<value_taken>` is the type of the value (eg int, char, float, signed, unsigned ...)
* `<value_range>` represent either a list of bounds, an upper or a lower bound, or None

In [3]:
# 3. create data format file

import numpy as np
from enum import Enum
import enum
import datetime

# the use of Enum classes will prevent incorrect combination of values
class QuantitativeDataType(Enum):
    CONTINUOUS = [float, np.float64]
    DISCRETE = [int, np.int64]

class CategoricalDataType(Enum):
    BOOLEAN = [bool]
    NUMERICAL = [float, int, np.float64, np.int64]
    CHARACTER = [str, object]
    
class KeyDataType(Enum):
    NUMERICAL = [int, np.int64]
    CHARACTER = [str, object]
    DATETIME = [pd.Timestamp,
                pd.Timedelta,
                pd.Period,
                datetime.datetime,
                np.datetime64]


class CustomDataType(Enum):
    """for demo purpose: here a custom datatype"""
    DISCRETE = [int, np.int64]
    CHARACTER = [str, object]
    
    
class DataType(Enum):
    """

    """
    # what about KEY = KeyDataType()
    KEY = [KeyDataType.NUMERICAL,
           KeyDataType.CHARACTER,
           KeyDataType.DATETIME]
    QUANTITATIVE = [QuantitativeDataType.CONTINUOUS,
                   QuantitativeDataType.DISCRETE]
    CATEGORICAL = [CategoricalDataType.BOOLEAN,
                  CategoricalDataType.NUMERICAL,
                  CategoricalDataType.CHARACTER]
    #MISSING = 'MISSING'
    #DATETIME = 'DATETIME'
    DATETIME = [pd.Timestamp,
                pd.Timedelta,
                pd.Period,
                datetime.datetime,
                np.datetime64]
    CUSTOM = [CustomDataType.DISCRETE,
             CustomDataType.CHARACTER]
    UNKNOWN = 'UNKNOWN'
    
    @staticmethod
    def get_names():
        return tuple(n for n, _ in DataType.__members__.items())

class MissingValueAllowedDefault(Enum):
    KEY = False
    QUANTITATIVE = True
    CATEGORICAL = True
    DATETIME = False
    
    @staticmethod
    def get_names():
        return tuple(n for n, _ in MissingValueAllowedDefault.__members__.items())
    
    
class DataTypeProperties(Enum):
    """Data Type possible modification (when editing)"""
    CATEGORICAL = (False, False, True, False)
    QUANTITATIVE = (True, True, False, False)
    DATETIME = (True, True, False, True)
    UNKNOWN = (False, False, False, False)
    CUSTOM = (True, True, True, False)
    
    def __init__(self,
                 lower_bound: bool,
                 upper_bound: bool,
                 set_of_values: bool,
                 date_format:bool):
        self._lower_bound = lower_bound
        self._upper_bound = upper_bound
        self._set_of_values = set_of_values
        self._date_format = date_format
        
    @property
    def lower_bound(self):
        return self._lower_bound
    
    @property
    def upper_bound(self):
        return self._upper_bound
    
    @property
    def set_of_values(self):
        return self._set_of_values
    
    @property
    def date_format(self):
        return self._date_format

In [4]:
def get_data_type(
                  #avail_data_types: enum.EnumMeta,
                  d_format: Enum,
                  d_type: type) ->  Tuple[Enum, List[Union[type, str]]]:
    present_d_types = []
    sub_d_type_format = None
    for avail_data_type in DataType:
        if d_format is avail_data_type:
            sub_dtypes = avail_data_type.value
            if not isinstance(sub_dtypes, str) and hasattr(sub_dtypes, '__getitem__') and isinstance(sub_dtypes[0], Enum):
                # check if dtype has subtypes
                #(eg if datatype is QUANTITATIVE, subtype will be CONTINOUS or DISCRETE)
                for sub_dtype in sub_dtypes:
                    if any(d_type == t for t in tuple(sub_dtype.value)):
                        present_d_types.append(d_type)
                        sub_d_type_format = sub_dtype
                        print(sub_dtype, d_type)
            else:
                # case where datatype doesnot have subtypes, eg DATETIME
                if any(d_type == t for t in sub_dtypes):
                    present_d_types.append(d_type)
                sub_d_type_format = d_format
    print(sub_d_type_format, '|', present_d_types)
    return  sub_d_type_format, present_d_types


def find_data_type(data_format_name: str, data_type_name: str=None) -> List[Enum]:
    """Retrieves from a given data_format and data_type,
    the corresponding Enum class describing data"""
    
    ## varible initialisation
    data_type = []
    _is_data_format_unrecognized = True
    _is_data_type_unrecognized = True
    
    _available_data_types = [t for t in DataType]
    
    for a_data_type in _available_data_types:
        if data_format_name == a_data_type.name:
            _is_data_format_unrecognized = False
            data_type.append(a_data_type)
            
            for sub_type in a_data_type.value:
                
                if data_type_name is not None and  not isinstance(sub_type, str):
                    # check if sub data type exist (it shouldnot if variable is UNKNOWN)
                    if data_type_name == sub_type.name:
                        
                        _is_data_type_unrecognized = False
                        data_type.append(sub_type)
                else:
                    _is_data_type_unrecognized = False
    # check for data formt file consistancy error
    if any((_is_data_format_unrecognized, _is_data_type_unrecognized)):
        if _is_data_format_unrecognized:
            print(f'error: {data_format_name} not recognized as a valid data type')
        else:
            print(f'error {data_type_name} not recognized as a valid data type')
            
    return data_type

In [5]:

def check_missing_data(column: pd.Series)->bool:
    is_missing_data = column.isna().any()
    return is_missing_data
df = pd.DataFrame({'w': [1, 2, 3, 4,  'jj', None]})
print(check_missing_data(df['w']))




True


CLI details:

1. open  csv file
2. for each columns in file ask type of variable or if variable should be excluded
3. automatically detect the type given values in columns 
4. ask for each columns if missing data are allowed


eg :

assume a column is of type discrete with integers


1. user select it is quantitative
2. then system will label it as quantitative-discrete


**Question** : do we want an auto selection parameter choice?

In [6]:
def get_yes_no_msg() -> str:
    msg_yes_or_no_question = '1) YES\n2) NO\n'   
    return msg_yes_or_no_question

def parse_yes_no_msg(resp: str) -> bool:
    """implements logic to parse yes or no msg"""
    yes_or_no_question_key = {'1': True,
                    '2': False}
    return yes_or_no_question_key.get(resp)

def get_data_type_selection_msg(available_data_type:List[Enum]) ->Tuple[str, int]:
    
    n_available_data_type = len(available_data_type)
    msg = ''

    
    for i, dtype in enumerate(available_data_type):
        msg += '%d) %s \n' %  (i+1, dtype.name)
    
    ignoring_key = i+2  # add ingoring entry
    msg += '%d) ignore this column\n' % (ignoring_key)
    
    
    
    return msg, ignoring_key

def unique(iterable: Iterator) -> int:
    """returns number of unique values"""
    return len(set(iterable))

In [69]:
# CLI for clinicians for setting up data format file



In [7]:
def get_from_user_dataframe_format_file(dataset: pd.DataFrame) -> Dict[str, Any]:
    
    dataset_columns = dataset.columns
    dataset_columns_length = len(dataset_columns)
    data_format_file = {}
    
    
    
    available_data_type = [d_type for d_type in DataType]  # get all available data types
    
    for n_feature, feature in enumerate(dataset_columns):
        print(f'displaying first 10 values of feature {feature} (n_feature: {n_feature+1}/{dataset_columns_length})')
        #print(tabulate(dataset[feature].head(10).values()))
        pprint.pprint(dataset[feature].head(10))  # print first 10 lines of feature value
        print(f'number of differents samples: {unique(dataset[feature])} / total of samples: {dataset[feature].shape[0]}')
        
        msg_data_type_selection, ignoring_id = get_data_type_selection_msg(available_data_type)
        msg_data_type_selection = f'specify data type for {feature}:\n' + msg_data_type_selection
        
        # ask user about data type
        data_format_id = get_user_input(msg_data_type_selection,
                                       
                                       n_answers=dataset_columns_length+1)
        
        if int(data_format_id) > ignoring_id - 1:
            # case where user decide to ingore column: go to next iteration (next feature)
            print(f"Ignoring feature {feature}")
            continue
        else:
            # case where user selected a data type: add data type and info to the format file
            data_format = available_data_type[int(data_format_id)-1]
            data_type = dataset[feature].dtype
            n_data_type, types = get_data_type(data_format, data_type)
            
        # KEY and DATETIME type 
        if data_format is DataType.KEY or data_format is DataType.DATETIME:  
            is_missing_values_allowed = False
        else: 
            # ask user if missing values are allowed for this specific variable
            msg_yes_or_no_question = get_yes_no_msg()
            msg_yes_or_no_question = f'Allow {feature} to have missing values:\n' + msg_yes_or_no_question
            missing_values_user_selection = get_user_input(msg_yes_or_no_question,
                                                        n_answers=2)
            is_missing_values_allowed = parse_yes_no_msg(missing_values_user_selection)
            
        if hasattr(n_data_type, 'name'):
            name_data_type = n_data_type.name
        else:
            name_data_type = n_data_type[0]
        data_format_file[feature] = {'data_format': data_format.name,
                                     'data_type': n_data_type.name,
                                     'values': str(data_type),
                                     'is_missing_values': is_missing_values_allowed}
            
    return data_format_file
            
def get_user_input(msg:str,  n_answers:int) -> str:
    """"""
    is_column_parsed = False
    while not is_column_parsed:
        #data_format_id = input(f'specify data type for {feature}:\n' + msg )
        resp = input(msg)
        if resp.isdigit() and int(resp) <= n_answers and int(resp)>0:
            # check if value passed by user is correct (if it is integer,
            # and whithin range [1, n_available_data_type])
            is_column_parsed = True

        else:
            print(f'error ! {resp} value not understood')
            
    return resp

In [8]:
### CLI to use when dataset is available


def get_from_user_multi_view_dataset_fromat_file(datasets: Dict[str, pd.DataFrame])-> Dict[str, pd.DataFrame]:
    
    data_format_files = {}
    
    for tabular_data_file in datasets.keys():
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print(f"+++++++ Now parsing view: {tabular_data_file} +++++++")
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        data_format_file = get_from_user_dataframe_format_file(datasets[tabular_data_file])
        
        _file_name = os.path.basename(tabular_data_file)
        data_format_files[_file_name] = data_format_file
        
    return data_format_files


In [14]:
data_format_file = get_from_user_multi_view_dataset_fromat_file(single_view_dataset)

++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++ Now parsing view: pseudo_adni_mod.csv +++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++
displaying first 10 values of feature CDRSB.bl (n_feature: 1/16)
0    1
1    0
2    0
3    0
4    0
5    1
6    4
7    0
8    3
9    2
Name: CDRSB.bl, dtype: int64
number of differents samples: 8 / total of samples: 1000
specify data type for CDRSB.bl:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) CUSTOM 
6) UNKNOWN 
7) ignore this column
6
DataType.UNKNOWN | []


KeyboardInterrupt: Interrupted by user

data_fromat_ref (read only)

CLI editer data_format_file

review : 
- specify lower / upper bound NUMERICAL
- Specify categorical (BOOLEAN, CHARACTER, NUMERICAL)

- save different categorical values 
 a posteriori ex SEX -> male or female, NOT FEMALE
- 

In [10]:
multi_data_format_file = get_from_user_multi_view_dataset_fromat_file(multi_view_dataframe)


NameError: name 'multi_view_dataframe' is not defined

In [115]:
multi_data_format_file

{'file1': {'e': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  '1': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': False},
  '2': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': True},
  'time': {'data_format': 'DATETIME',
   'data_type': 'DATETIME',
   'values': 'object',
   'is_missing_values': False},
  'pressure': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'e.1': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'gender': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': False},
  'blood type': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': True},
  '

In [116]:
save_format_file_ref(multi_data_format_file, 'multi_format_file')

Model successfully saved at multi_format_file


In [48]:
# saving data format file

json_file_name = "format_file_ref"

with open(json_file_name, "w") as format_file:
    json.dump(data_format_file, format_file)

In [85]:
def save_format_file_ref(format_file_ref: Dict[str, Dict[str, Any]], path: str):
    # save `format_file_ref` into a JSON file
    with open(path, "w") as format_file:
        json.dump(format_file_ref, format_file)
    print(f"Model successfully saved at {path}")

In [91]:
def load_format_file_ref(path: str) -> Dict[str, Dict[str, Any]]:
    # retrieve data format file
    with open(path, "r") as format_file:
        format_file_ref = json.load(format_file)
    return format_file_ref

In [92]:

json_file_name = "format_file_ref"


format_file = load_format_file_ref(json_file_name)

In [93]:
format_file

{'pseudo_adni_mod.csv': {'CDRSB.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'int64',
   'is_missing_values': True},
  'ADAS11.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  'MMSE.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'RAVLT.immediate.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': True},
  'RAVLT.learning.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'FAQ.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'TAU.MEDIAN.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'AGE': {'data_format': 'CATEGORICAL',
   'data_t

In [13]:
create_msg_action_selection(DataTypeProperties.CUSTOM)

('1)data_type2)lower bound3)upper bound4) Values taken5) Cancel Operation', 5)

In [12]:
def create_msg_action_selection(data_type_propreties: Enum) -> Tuple[str, int]:
    # create edit selection message for user given data type
    msg = ""
    action_counter = 1
    # special case for data_type DATETIME
    if data_type_propreties.name != DataType.DATETIME.name:
        msg += "%d)data_type\n" % action_counter
        action_counter += 1
        
    if data_type_propreties.lower_bound:
        msg += "%d)lower bound\n" % action_counter
        action_counter += 1
    
    if data_type_propreties.upper_bound:
        msg += "%d)upper bound\n" % action_counter
        action_counter += 1
        
    if data_type_propreties.set_of_values:
        msg += "%d) Values taken\n" % action_counter
        action_counter += 1
        
    if data_type_propreties.date_format:
        msg += "%d) Date format\n" % action_counter
        action_counter += 1
    
    msg += "%d) Cancel Operation\n" % action_counter
    
    return msg, action_counter

def select_action(data_format: str,
                  action: str,
                  available_categorical_data_type: List[Enum],
                  msg:str) -> Tuple[Dict[str, Any], bool]:
    
    # variable initialization
    is_cancelled = False
    _action_counter = 1  # integer for dynamic cli management
    _avail_data_type_properties = [dtype for dtype in DataTypeProperties]
    
    for data_type_properties in avail_data_type_properties:
        if data_format == data_type_properties.name:
            # define action
            # action for lower bound
            if data_type_properties.lower_bound:
                pass
    if data_format == DataType.CATEGORICAL.name:
        
        if action == '1':
            new_field = ask_for_data_type(available_categorical_data_type,
                                          msg)
        elif action =='2':
            new_field = ask_for_categorical_values()
        elif action == '3':
            print('operation cancelled')
            new_field = None
            is_cancelled = True
            
        else:
            
            new_field = None
    else:
        # for QUANTITATIVE, DATETIME and KEY data types
        
        if action == '1':
            new_field =  ask_for_lower_bound()
        elif action == '2':
            new_field = ask_for_upper_bound()
        elif action == '3':
            new_field = None
            print('operation cancelled')
            is_cancelled = True
        else:
            new_field = None
    
    return new_field, is_cancelled


def isfloat(value:str) ->bool:
    """checks if string represents a float or int"""
    is_float = True
    try:
        float(value)
    except ValueError as e:
        is_float = False
    return is_float
    
        
def ask_for_lower_bound() -> Dict[str, float]:
    _is_entered_value_correct = False
    while not _is_entered_value_correct:
        lower_bound = input('enter lower bound')
        if isfloat(lower_bound):
            # check if entered value is correct (is a numerical value)
            _is_entered_value_correct = True
    return {'lower_bound': float(lower_bound)}

def ask_for_upper_bound() -> Dict[str, float]:
    
    _is_entered_value_correct = False
    while not _is_entered_value_correct:
        upper_bound = input('enter upper bound')
        if isfloat(upper_bound):
            # check if entered value is correct (is a numerical value)
            _is_entered_value_correct = True
    return {'upper_bound': float(upper_bound)}

def ask_for_data_type(
                      available_categorical_data_type: List[Enum], 
                      msg: str) -> Dict[str, Any]:
    # edit categorical datatype
    data_type_selection = get_user_input(msg, 4)
    updates = None
    if data_type_selection != '4':
        new_data_type = available_categorical_data_type[int(data_type_selection) - 1]
        new_values = list(map(lambda x: str(x), new_data_type.value))
        updates = {'data_type': new_data_type.name, 'values': new_values}
    return updates


def ask_for_categorical_values() -> Dict[str, Any]:
    possible_values = input('enter possible values (separated by ",")')
    possible_values = possible_values.split(",")  # separate values passed by user into a list
    return {'categorical_values': possible_values}



def edit_feature_format_file_ref(feature_content: Dict[str, Any],
                                  feature_name: str,
                                  available_categorical_data_type: List[Enum],
                                  messages: Dict[str, str],
                                  ignore_keystroke: int) -> Dict[str, Any]:
    """Edits a specific feature that belongs to a specific view within a format file"""
    

    _is_feature_unparsed = True  
    _is_cancelled = False  # whether parsing of current column has been cancelled or not
    _is_first_edit = True
    
    # iterate over number of feature contained in view, and ask for each feature if changes are needed
    while _is_feature_unparsed:
        if _is_cancelled or not _is_first_edit:
            _f_answer = True
        else:
            _f_answer = get_user_input(f"Edit variable: {feature_name}?\n" + messages['yes_or_no'], 2)
            # ask if user wants to edit feature names
            _f_answer = parse_yes_no_msg(_f_answer)
            _is_operation_cancelled = False  # for cancelling feature edition
            _is_first_edit = False
        if _f_answer:
            # case where user wants to edit the current feature
            
            _msg = messages['edit']
            if feature_content.get('data_format') == DataType.CATEGORICAL.name:
                # case if feature is a categorical variable
                 _msg += messages['categorical_edit']
            else:
                #case if feature is a quantitative variable
                _msg += messages['quantitative_edit']
                
            _msg += messages['ignore']
            _edit_selection = get_user_input(_msg, 3)
            
            _edited_field, _is_cancelled = select_action(feature_content.get('data_format'),
                                                          _edit_selection,
                                                          available_categorical_data_type,
                                                          messages['data_type_select'])
            
            if not _is_cancelled:
                # if user has not cancelled field edition
                if _edited_field is not None:
                    feature_content.update(_edited_field)
             
                _c_answer = get_user_input(f"Continue Editing variable: {feature_name}?\n" + messages['yes_or_no'], 2)
                _is_feature_unparsed = parse_yes_no_msg(_c_answer)
            else:
                _is_feature_unparsed = False
                
        else:
            _is_feature_unparsed = False
            
    return feature_content

In [66]:
def edit_format_file_ref(format_file_ref: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    
    # CLI for editing `format_file_ref`, a file containing information about each variable
    # in a tabular dataset
    print(f'Now editing format file ref')
    
    ## variables initialization
    available_categorical_data_types = [t for t in CategoricalDataType]
    _file_names = list(format_file_ref.keys())
    _n_tot_files = len(_file_names)
    
    ## messages definition
    _data_type_selection_msg, ign_key = get_data_type_selection_msg(available_categorical_data_types)
    
    _messages = {
        'yes_or_no': get_yes_no_msg(),
        'data_type_select':  _data_type_selection_msg,
        'edit': 'Which field should be modified?\n',
        'quantitative_edit': '1)lower_bound\n2)upper_bound\n',
        'categorical_edit': '1)data_type (categorical variable only)\n2)values taken (categorical variable only)\n',
        'ignore': '3)cancel operation\n'
    }

    
    
    # iterate over name of files (ie views)
    for i_file in range(_n_tot_files):
        # ask for each file if user wants to edt it
        _answer = get_user_input(f"Edit file: {_file_names[i_file]}?\n" + _messages['yes_or_no'], 2)
        _answer = parse_yes_no_msg(_answer)
        
        if _answer:
            # case where user wants to modify current view scheme
            _file_content = format_file_ref[_file_names[i_file]]  # get file (ie view) content
            
            ## variables initialization for parsing current view
            _features_names = list(_file_content.keys())
            _n_tot_feature = len(_features_names)
            
            # iterate over features found in view
            for i_feature in range(_n_tot_feature):
                feature_name = _features_names[i_feature]
                feature_content = _file_content[feature_name]
                feature_content = edit_feature_format_file_ref(feature_content,
                                                               feature_name,
                                                               available_categorical_data_types,
                                                               _messages,
                                                               ign_key)
            format_file_ref[_file_names[i_file]].update({feature_name: feature_content})
            
    return format_file_ref

In [90]:
format_file

{'pseudo_adni_mod.csv': {'CDRSB.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': ["<class 'bool'>"],
   'is_missing_values': True,
   'categorical_values': ['1', ' 2', ' 4']},
  'ADAS11.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  'MMSE.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'RAVLT.immediate.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': True},
  'RAVLT.learning.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'FAQ.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False,
   'lower_bound': 0.0},
  'TAU.MEDIAN.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is

In [ ]:
edit_format_file_ref(format_file)

# TODO: create possiblity for user to update data type when editing format_file

In [96]:
multi_format_file_ref = load_format_file_ref('multi_format_file')
edit_format_file_ref(multi_format_file_ref)

Now editing format file ref
Edit file: file1?
1) YES
2) NO
1
Edit variable: a?
1) YES
2) NO
1
Which field should be modified?
1)lower_bound
2)upper_bound
3)cancel operation
1
enter lower bound30
Continue Editing variable: a?
1) YES
2) NO
1
Which field should be modified?
1)lower_bound
2)upper_bound
3)cancel operation
2
enter upper bound2
Continue Editing variable: a?
1) YES
2) NO
2
Edit variable: 0?
1) YES
2) NO
2
Edit variable: pressure?
1) YES
2) NO
1
Which field should be modified?
1)lower_bound
2)upper_bound
3)cancel operation
3
operation cancelled
Edit variable: sp02?
1) YES
2) NO
2
Edit variable: i.1?
1) YES
2) NO
2
Edit variable: gender?
1) YES
2) NO
1
Which field should be modified?
1)data_type (categorical variable only)
2)values taken (categorical variable only)
3)cancel operation
2
enter possible values (separated by ",")MALE, FEMALE
Continue Editing variable: gender?
1) YES
2) NO
2
Edit variable: blood type?
1) YES
2) NO
2
Edit file: contatct?
1) YES
2) NO
2
Edit file: file

{'file1': {'a': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True,
   'lower_bound': 30.0,
   'upper_bound': 2.0},
  '0': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': True},
  'pressure': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'sp02': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'i.1': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'int64',
   'is_missing_values': False},
  'gender': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': True,
   'categorical_values': ['MALE', ' FEMALE']},
  'blood type': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': False}},
 'contatct': {'discr

## tabular data sanity check using file format ref

In [19]:
# utility functions for multi view dataframe
def rename_variables_before_joining(multi_view_datasets: Dict[str, pd.DataFrame],
                                    views_name: List[Union[str, int]],
                                    primary_key:Union[str, int]=None) -> Dict[str, pd.DataFrame]:
    """
    Renames variables that have same name but different views using the following naming convention:
    if `a` is the name of a feature of `view1` and `a` is the name of a feature of `view2`,
    features names will be updated into `view1.a` and `view2.a`
    """
    _features_names = {}
    _views_length = len(views_name)
    
    # parse every combination once
    for i_left in range(0, _views_length-1):
        _left_view = views_name[i_left]
        _left_features_name = multi_view_datasets[_left_view].columns.tolist()
        for i_right in range(i_left+1, _views_length):
        
            _right_view = views_name[i_right]
            _right_features_name = multi_view_datasets[_right_view].columns.tolist()
            
            for _f in _left_features_name:
                if primary_key and _f == primary_key:
                    # do not affect primary key (if any)
                    continue
                if _f  in _right_features_name:
                    
                    if _left_view  not in _features_names:
                        _features_names[_left_view] = {}
                        
                    if _right_view not in _features_names:
                        _features_names[_right_view] = {}
                        
                    _features_names[_left_view].update({_f: _left_view + '.' + str(_f)})
                    _features_names[_right_view].update({_f: _right_view + '.' + str(_f)})
    
    for i in range(_views_length):
        _view = views_name[i]
        _new_features = _features_names.get(_view)
        if _new_features:
            multi_view_datasets[_view] = multi_view_datasets[_view].rename(columns=_new_features)
        
    
    return multi_view_datasets


def create_multi_view_dataframe(datasets: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    _header_labels = ['views', 'feature_name']
    # 1. create multiindex header

    _feature_name_array = np.array([])  # store all feature names
    _view_name_array = []  # store all views (ie modalities) names

    _concatenated_datasets = np.array([])  # store dataframe values

    for key in datasets.keys():
        _feature_name_array = np.concatenate([_feature_name_array,
                                              datasets[key].columns.values])
        if len(_concatenated_datasets) <= 0:
            # first pass 
            _concatenated_datasets = datasets[key].values
        else:
            # next passes
            try:
                _concatenated_datasets = np.concatenate(
                                        [_concatenated_datasets,
                                         datasets[key].to_numpy()
                                         ], axis=1)
            except ValueError as val_err:
                # catching case where nb_samples are differents
                raise ValueError(
                    'Cannot create multi view dataset: different number of samples for each modality have been detected'\
                        + 'Details: ' + str(val_err)
                    )
        for _ in datasets[key].columns.values:
            _view_name_array.append(key)

    _header = pd.MultiIndex.from_arrays([_view_name_array,
                                         _feature_name_array],
                                        names=_header_labels)


    # 2. create multi index dataframe

    multi_view_df = pd.DataFrame(_concatenated_datasets,
                                  columns = _header)
    return multi_view_df


def join_muti_view_dataset(multi_view_dataset: pd.DataFrame,
                           primary_key: str=None) -> pd.DataFrame:
    """Concatenates a multi view dataset into a plain pandas dataframe,
    by doing a join operation along specified primary_key"""
    _views_name = sorted(set(multi_view_dataset.columns.get_level_values(0)))  # get views name
    
    joined_dataframe = multi_view_dataset[_views_name[0]]  # retrieve the first view
    # (as a result of join operation)
    for x in range(1, len(_views_name)):
        joined_dataframe = joined_dataframe.merge(multi_view_dataset[_views_name[x]],
                                                    on=primary_key,
                                                    suffixes=('', '.'+_views_name[x]))
        
        #df['file1'].join(df['file2'].set_index('pkey'), on='pkey', rsuffix='.file2')
        
    return joined_dataframe


def search_primary_key(format_file_ref: Dict[str, Dict[str, Any]]) -> Optional[str]: 
    """"""
    _views_names = list(format_file_ref.keys())
    primary_key = None
    _c_view = None
    for view_name in _views_names:
        file_content = format_file_ref[view_name]
        _features_names = list(file_content.keys())
        for feature_name in _features_names:
            feature_content  = file_content[feature_name]
            _d_format = feature_content.get('data_format')
            
            if _d_format == DataType.KEY.name:
                if _c_view is None:
                    primary_key = feature_name
                    _c_view = view_name
                    print(f'found primary key {primary_key}')
                else:
                    print(f'error: found 2 primary keys is same view {view_name}')
        _c_view = None
    return primary_key



def select_data_from_format_file_ref(datasets: Dict[str, Dict[str, Any]],
                                     format_file: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """returns an updated dataset containing only the features detailed in format_file"""
    # variables initialisation
    
    updated_dataset = {}
    _views_format_file = list(format_file.keys())
    
    for view in _views_format_file:
        if view in datasets.keys():
            # only extract features from format_file
            _format_file_features = list(format_file[view].keys())
            _current_dataset_feature = datasets[view].columns.tolist()
            try:
                updated_dataset[view] = datasets[view][_format_file_features]
            except KeyError as ke:
                # catch error if a column is specified in data format file
                # but not found in dataset
                _missing_feature = []
                for feature in _format_file_features:
                    if feature not in _current_dataset_feature:
                        _missing_feature.append(feature)
                print('Error: th following features', *_missing_feature, f'are not found in view: {view}')
        else:
            # trigger error
            print(f'error!: missing view {view} in dataset')
            
    return updated_dataset

In [20]:
dataset_to_check = load_tabular_datasets(r'/user/ybouilla/home/Documents/data/pseudo_adni_mod/pseudo_adni_mod.csv')



file found


In [22]:
# extract views names
views_names = list(format_file.keys())



# look for primary key
primary_key = search_primary_key(format_file)
print('primary key', primary_key)

# select only features in dataset that will be checked
pre_parsed_dataset_to_check = select_data_from_format_file_ref(dataset_to_check, format_file)
# rename columns names before join operation
pre_parsed_dataset_to_check = rename_variables_before_joining(pre_parsed_dataset_to_check, views_names)
pre_parsed_dataset_to_check

multi_df_to_check = create_multi_view_dataframe(pre_parsed_dataset_to_check)
multi_df_to_check

#if primary_key is not None:
# jointure operation (takesplace only if primary key has been specfied in foramt_file)
df_to_check = join_muti_view_dataset(multi_df_to_check)
    
df_to_check

primary key None


feature_name,CDRSB.bl,ADAS11.bl,MMSE.bl,RAVLT.immediate.bl,RAVLT.learning.bl,FAQ.bl,TAU.MEDIAN.bl,AGE
0,1.0,8.0,27.0,23.739439,4.0,3.0,132.571916,75.0
1,0.0,0.0,30.0,64.933800,9.0,0.0,33.787719,67.0
2,0.0,8.0,24.0,36.987722,3.0,0.0,110.049924,63.0
3,0.0,3.0,29.0,50.314425,5.0,3.0,138.690457,75.0
4,0.0,0.0,30.0,57.217830,9.0,0.0,-61.573234,65.0
...,...,...,...,...,...,...,...,...
995,1.0,2.0,29.0,61.896022,8.0,0.0,87.065806,76.0
996,0.0,1.0,29.0,62.083170,8.0,1.0,121.985248,77.0
997,3.0,14.0,24.0,22.289059,2.0,7.0,157.229102,74.0
998,0.0,13.0,26.0,31.650504,2.0,4.0,103.238647,64.0


In [80]:
# sanity check on columns using format_file



_views_format_file = list(format_file.keys())
    
for view in _views_format_file:
    _features_format_file = format_file[view]
    
    for feature in _features_format_file:
        feature_content = df_to_check[feature]
        feature_format =  _features_format_file[feature]
        # 1. check for datatype consistency
        
        data_types = find_data_type(feature_format['data_format'], feature_format['data_type'])
        
        print(dtype[-1].value, feature_content.dtype, feature_format)
        if any(t == feature_content.dtype for t in data_types[-1].value ):
            print('ok')
        


[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'CATEGORICAL', 'data_type': 'BOOLEAN', 'values': ["<class 'bool'>"], 'is_missing_values': True, 'categorical_values': ['1', ' 2', ' 4']}
[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'QUANTITATIVE', 'data_type': 'DISCRETE', 'values': 'int64', 'is_missing_values': True}
[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'CATEGORICAL', 'data_type': 'NUMERICAL', 'values': 'float64', 'is_missing_values': False}
ok
[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'QUANTITATIVE', 'data_type': 'CONTINUOUS', 'values': 'float64', 'is_missing_values': True}
ok
[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'CATEGORICAL', 'data_type': 'NUMERICAL', 'values': 'float64', 'i

[<DataType.CATEGORICAL: [<CategoricalDataType.BOOLEAN: [<class 'bool'>]>, <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>, <CategoricalDataType.CHARACTER: [<class 'str'>, <class 'object'>]>]>,
 <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>]

## data_format_file

In [33]:
format_file

{'pseudo_adni_mod.csv': {'CDRSB.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'int64',
   'is_missing_values': True},
  'ADAS11.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  'MMSE.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'RAVLT.immediate.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': True},
  'RAVLT.learning.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'FAQ.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'TAU.MEDIAN.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'AGE': {'data_format': 'CATEGORICAL',
   'data_t

In [33]:
msg_yes_or_no_question

'1) YES\n2) NO\n'

In [40]:
is_views_finished = False


views_format_file = {}

while not is_views_finished:
    is_features_finished = False
    resp = input('do you want to add a new view (file)?\n' + msg_yes_or_no_question)
    resp = yes_or_no_question_key.get(resp)
    if not resp:
        is_views_finished = True
        print('process done')
        continue
    new_view = input('please add new view name:\n')
    while not is_features_finished:
        feature_format_file = {}
        new_feature = input('please add new feature name:\n')
        feature_format_file[new_feature] = {}
        is_column_parsed = False
        try:
            while not is_column_parsed:
                data_format_id = input(f'specify data type for {feature}:\n' + msg )
                if data_format_id.isdigit() and int(data_format_id) <= n_available_data_type+1:
                    # check if value passed by user is correct (if it is integer,
                    # and whithin range [1, n_available_data_type])
                    is_column_parsed = True
                
                else:
                    print(f'error ! {data_format_id} value not understood')
                    
        except KeyboardInterrupt as e:
            print('stopping now' + str(e))
        resp = input('do you want to add a new variable (feature) ?' + msg_yes_or_no_question)
        resp = yes_or_no_question_key.get(resp)
        if not resp:
            is_features_finished = True
            print('process done')
            continue
    views_format_file[new_view] = feature_format_file

do you want to add a new view (file)?
1) YES
2) NO
1
please add new view name:
ll
please add new feature name:
ll
specify data type for 0:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) UNKNOWN 
6) ignore this column
4
do you want to add a new variable (feature) ?1) YES
2) NO
2
process done
do you want to add a new view (file)?
1) YES
2) NO
1
please add new view name:
kk
please add new feature name:
vkeof
specify data type for 0:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) UNKNOWN 
6) ignore this column
3
do you want to add a new variable (feature) ?1) YES
2) NO
2
process done
do you want to add a new view (file)?
1) YES
2) NO
2
process done


In [75]:
type(np.datetime64("2018-01-01"))
import datetime
type(datetime.datetime(2018, 1, 1))

pd.datetime64[ns]

AttributeError: module 'pandas' has no attribute 'datetime64'

In [91]:
t = type(pd.to_datetime('13000101', format='%Y%m%d', errors='ignore'))

In [107]:
t = pd.Series(pd.date_range("1/1/2011", freq="H", periods=3)).dtype

t =type(t).type

In [108]:
any(t == t1 for t1 in [pd.Timestamp, pd.Timedelta, pd.Period, datetime.datetime,np.datetime64] )

True

In [105]:
t.type

numpy.datetime64

In [28]:
data_format_file

{'CDRSB.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': int,
  'is_missing_values': False},
 'ADAS11.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': int,
  'is_missing_values': False},
 'MMSE.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': int,
  'is_missing_values': False},
 'RAVLT.immediate.bl': {'data_type': <QuantitativeDataType.CONTINUOUS: [<class 'float'>, <class 'numpy.float64'>]>,
  'values': float,
  'is_missing_values': False},
 'RAVLT.learning.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': float,
  'is_missing_values': False},
 'RAVLT.forgetting.bl': {'data_type': <QuantitativeDataType.CON

In [29]:

type(dataset[feature].dtype)

numpy.dtype[float64]

In [28]:
dir(dataset[feature])

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__r